In [1]:
import random
import pandas as pd

In [2]:
df = pd.read_csv('data/adventures-list.csv')
df = df.dropna(subset=['Adventure'])
df['Base Rarity'] = df['Base Rarity'].astype('int')
df['Release'] = df['Release'].astype('int')
df.head()

,Adventure,Base Rarity,Class,Weapon,Element,Availability,Gender,Release
0,Taiga,2,Warrior,Dagger,Ruby,Story,Female,0
1,Khun,2,Cleric,Blade,Sapphire,Story,Male,0
2,Rawn,2,Ranger,Axe,Emerald,Story,Male,0
3,Ryujin,2,Rogue,Lance,Topaz,Story,Male,0
4,Kaede,2,Knight,Staff,Onyx,Story,Female,0


In [3]:
RATES = {1: 0.75, 2: 0.2, 3: 0.05}
FEATURED = {2: 0.04, 3: 0.01}

### Free Adventures

In [4]:
adv = df.loc[df['Release'] == 0]
adv['Adventure'].unique().tolist()

['Taiga', 'Khun', 'Rawn', 'Ryujin', 'Kaede', 'Sayori']

### Initial Rolls

In [5]:
def simulate_roll(num, release):
    result = pd.DataFrame()
    for n in range(num):
        res = roll(release)
        if res is not None:
            result = result.append(res)
    return result

def roll(release, pity=False):
    pool = df.loc[df['Release'] <= release]
    die1 = random.random()
    if die1 <= RATES[1]:
        return
    elif die1 <= RATES[1] + RATES[2]:
        die2 = random.random()
        adv = None
        if die2 <= FEATURED[2] / RATES[2]:
            adv = get_featured(2, release, pool)
        if adv is None:
            adv = get_random_adventure(2, pool)
        return adv
    else:
        die2 = random.random()
        if die2 <= FEATURED[3] / RATES[3]:
            adv = get_featured(3, release, pool)
        else:
            adv = get_random_adventure(3, pool)
        return adv
            
def get_random_adventure(rarity, pool):
    return pool.loc[pool['Base Rarity'] == rarity].sample()

def get_featured(rarity, release, pool):
    row = pool.loc[(pool['Base Rarity'] == rarity) & (pool['Release'] == release)]
    if len(row) < 1:
        return None
    elif len(row) > 1:
        row = row.sample()
    return row

In [6]:
roll(1)

In [7]:
simulate_roll(30, 1)

,Adventure,Base Rarity,Class,Weapon,Element,Availability,Gender,Release
7,Damian,2,Ranger,Gun,Ruby,Permanent,Male,1
23,Rea,2,Ranger,Bow,Topaz,Permanent,Female,1
33,Eres,2,Ranger,Dagger,Amber,Permanent,Female,1
2,Rawn,2,Ranger,Axe,Emerald,Story,Male,0
16,Kiris,2,Knight,Wand,Emerald,Permanent,Female,1
4,Kaede,2,Knight,Staff,Onyx,Story,Female,0
22,Ryza,2,Knight,Gun,Topaz,Permanent,Female,1


In [8]:
simulate_roll(100, 1)['Base Rarity'].value_counts()

2    17
3     5
Name: Base Rarity, dtype: int64

In [9]:
simulate_roll(10, 2)

,Adventure,Base Rarity,Class,Weapon,Element,Availability,Gender,Release
21,Ozanne,2,Warrior,Axe,Topaz,Permanent,Female,1
26,Udo,2,Warrior,Sword,Onyx,Permanent,Female,1


In [10]:
simulate_roll(10, 3)

,Adventure,Base Rarity,Class,Weapon,Element,Availability,Gender,Release
31,Devon,2,Warrior,Mace,Amber,Permanent,Male,1
35,Miki,3,Cleric,Gun,Amber,Permanent,Female,1
5,Sayori,2,Mage,Wand,Amber,Story,Female,0
32,Phillip,2,Knight,Lance,Amber,Permanent,Male,1


In [11]:
simulate_roll(10, 15)

,Adventure,Base Rarity,Class,Weapon,Element,Availability,Gender,Release
7,Damian,2,Ranger,Gun,Ruby,Permanent,Male,1
33,Eres,2,Ranger,Dagger,Amber,Permanent,Female,1
54,Cotton,3,Cleric,Wand,Emerald,Permanent,Male,11


In [12]:
def get_roll_average(num_samples, num_rolls, release):
    stars2 = 0
    stars2f = 0
    stars3 = 0
    stars3f = 0
    for n in range(num_samples):
        r = simulate_roll(num_rolls, release)
        stars2 += len(r.loc[(r['Base Rarity'] == 2) & (r['Release'] != release)])
        stars2f += len(r.loc[(r['Base Rarity'] == 2) & (r['Release'] == release)])
        stars3 += len(r.loc[(r['Base Rarity'] == 3) & (r['Release'] != release)])
        stars3f += len(r.loc[(r['Base Rarity'] == 3) & (r['Release'] == release)])
    stars2 = round(stars2/num_samples, 2)
    stars2f = round(stars2f/num_samples, 2)
    stars3 = round(stars3/num_samples, 2)
    stars3f = round(stars3f/num_samples, 2)
    return stars2, stars2f, stars3, stars3f

In [13]:
get_roll_average(200, 100, 2)

(15.95, 4.44, 3.38, 1.58)

In [14]:
get_roll_average(200, 100, 15)

(20.04, 0.0, 3.8, 1.39)

In [15]:
def roll_till_feature(num_samples, release):
    rolls = []
    featured = df.loc[(df['Base Rarity'] == 3) & (df['Release'] == release), 'Adventure'].item()
    for n in range(num_samples):
        r = 0
        while True:
            r += 1
            adv = roll(release)
            if adv is not None:
                if adv['Adventure'].item() == featured:
                    rolls.append(r)
                    break
    return rolls

In [16]:
roll_till_feature(10, 2)

[34, 39, 75, 7, 50, 61, 17, 34, 42, 172]

In [17]:
rolls = roll_till_feature(200, 2)
sum(rolls) / len(rolls)

67.58